In [1]:
%%configure -f
{ "conf":{
    "spark.pyspark.python": "python3",
    "spark.pyspark.virtualenv.enabled": "true",
    "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
    "spark.pyspark.virtualenv.type":"native",
    "spark.dynamicAllocation.enabled":"true",
    "spark.shuffle.service.enabled":"true",
    "spark.dynamicAllocation.minExecutors":"8",
    "spark.sql.sources.partitionOverwriteMode":"dynamic",
    "spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation":"true"
}
}

In [2]:
sc.install_pypi_package("pandas")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1617541905191_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("1_etl").enableHiveSupport().getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from datetime import datetime

import pyspark.sql.functions as F
import pyspark.sql.types as T

import pandas as pd
import numpy as np

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Collect some general infos on all the matches (game mode, platform, date , number of rounds)
query = """
SELECT 
matchid
,platform
,mapname
,gamemode
,MAX(roundnumber) AS max_roundnumber
,COUNT(*) AS count_rows
,MIN(dateid) AS dateid
FROM r6s_datapeek.dump_rounds
GROUP BY matchid, platform, mapname, gamemode
"""

dfs_match_stats = spark.sql(query).persist()
dfs_match_stats = dfs_match_stats.withColumn('max_roundnumber', F.col('max_roundnumber').cast(T.IntegerType()))
dfs_match_stats.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1592232

In [8]:
dfs_match_stats.sample(False, 0.1, 0).limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      matchid platform           mapname  ... max_roundnumber  count_rows    dateid
0  1011024541      PS4             HOUSE  ...               4          35  20170512
1  1016782541       PC  KAFE DOSTOYEVSKY  ...               4          40  20170212
2  1020114761     XONE             PLANE  ...               6          49  20170306
3  1022613961     XONE           FAVELAS  ...               5          50  20170321
4  1026015101     XONE            OREGON  ...               6          59  20170221

[5 rows x 7 columns]

In [9]:
# Collect the settings (configuration of the team) of the last round of each match
query = """
SELECT
t1.matchid
,t1.team
,t1.skillrank
,t1.clearancelevel
FROM r6s_datapeek.dump_rounds t1
INNER JOIN (
    SELECT
    matchid
    ,MAX(roundnumber) AS max_roundnumber
    FROM r6s_datapeek.dump_rounds
    GROUP BY matchid
) t2
ON t1.matchid == t2.matchid
AND t1.roundnumber == t2.max_roundnumber
"""

dfs_lastround_configuration = spark.sql(query).persist()
dfs_lastround_configuration = dfs_lastround_configuration.withColumn('user_attributes', F.array(F.col('skillrank'), F.col('clearancelevel')))# Build  tuple for each user with his level and rank
dfs_lastround_configuration.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14359666

In [10]:
# Aggregate the informations at the team level 
dfs_lastround_configuration_agg_team0 = dfs_lastround_configuration.filter(F.col('team') == '0').groupBy('matchid').agg(F.collect_list('user_attributes').alias('team0_user_attributes_lastround'))
dfs_lastround_configuration_agg_team1 = dfs_lastround_configuration.filter(F.col('team') == '1').groupBy('matchid').agg(F.collect_list('user_attributes').alias('team1_user_attributes_lastround'))
dfs_lastround_configuration_agg = dfs_lastround_configuration_agg_team0.join(dfs_lastround_configuration_agg_team1, on='matchid')
dfs_lastround_configuration_agg.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1583289

In [11]:
# Do the same but for the first round (with a small twist)
query = """
SELECT
matchid
,team
,role
,skillrank
,clearancelevel
FROM r6s_datapeek.dump_rounds
WHERE roundnumber = '1'
"""

dfs_firstround_configuration = spark.sql(query).persist()
dfs_firstround_configuration = dfs_firstround_configuration.withColumn('user_attributes', F.array(F.col('skillrank'), F.col('clearancelevel')))
dfs_firstround_configuration.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15198384

In [25]:
# Determine the role of each team at the begiining of the match
dfs_firstround_team_role = dfs_firstround_configuration.groupBy(F.col('matchid')).pivot('team').agg(F.first('role'))
dfs_firstround_team_role = dfs_firstround_team_role.select('matchid',F.col('0').alias('team0_role_firstround'), F.col('1').alias('team1_role_firstround'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
dfs_firstround_configuration_agg_team0 = dfs_firstround_configuration.filter(F.col('team') == '0').groupBy('matchid').agg(F.collect_list('user_attributes').alias('team0_user_attributes_firstround'))
dfs_firstround_configuration_agg_team1 = dfs_firstround_configuration.filter(F.col('team') == '1').groupBy('matchid').agg(F.collect_list('user_attributes').alias('team1_user_attributes_firstround'))
dfs_firstround_configuration_agg = dfs_firstround_team_role.join(dfs_firstround_configuration_agg_team0.join(dfs_firstround_configuration_agg_team1, on='matchid'), on='matchid')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# Join the infos of the first and last rounds
dfs_match_round_configuration = dfs_firstround_configuration_agg.join(dfs_lastround_configuration_agg, on='matchid').persist()
dfs_match_round_configuration.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1583065

In [28]:
def prepare_levels(infos, idx):
    levels = sorted([elt[idx] for elt in infos])
    return levels
    
def determine_team_changed(firstround, lastround):
    def determine_team_changed_(firstround, lastround):
        if sorted(firstround) != sorted(lastround):
                return 1
        return 0
    return F.udf(determine_team_changed_, T.IntegerType())(firstround, lastround)

# Compute a few metrics to evaluate the evolution of the teams between the first and last rounds (compare the level/rank, the delta of the size of the teams, and the size at the beginning)
for teamid in range(2):
    dfs_match_round_configuration = dfs_match_round_configuration.withColumn(f'team{teamid}_changed', determine_team_changed(F.col(f'team{teamid}_user_attributes_firstround'), F.col(f'team{teamid}_user_attributes_lastround')))
    dfs_match_round_configuration = dfs_match_round_configuration.withColumn(f'team{teamid}_deltasize', F.size(F.col(f'team{teamid}_user_attributes_lastround')) - F.size(F.col(f'team{teamid}_user_attributes_firstround')))
    dfs_match_round_configuration = dfs_match_round_configuration.withColumn(f'team{teamid}_startsize', F.size(F.col(f'team{teamid}_user_attributes_firstround')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Determne the number of rounds won by both teams
query = """
SELECT
matchid
,roundnumber
,team
,haswon
FROM r6s_datapeek.dump_rounds
GROUP BY matchid, roundnumber, team, haswon
"""
dfs_win_round = spark.sql(query)
dfs_win_round = dfs_win_round.withColumn('haswon', F.col('haswon').cast(T.IntegerType()))
dfs_win_match = dfs_win_round.groupBy(['matchid', 'team']).agg(F.sum('haswon').alias('count_round_won')).persist()
dfs_win_match = dfs_win_match.groupBy(F.col('matchid')).pivot('team').agg(F.first('count_round_won'))
dfs_win_match = dfs_win_match.select('matchid',F.col('0').alias('team0_role_count_round_won'), F.col('1').alias('team1_role_count_round_won'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
dfs_win_match.sample(False, 0.1, 0).limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      matchid  team0_role_count_round_won  team1_role_count_round_won
0   195260441                           2                           4
1  3692232409                           4                           1
2  1853762881                           0                           4
3  1708168241                           4                           1
4  3518396009                           4                           5

In [31]:
# Compute the number fo rows available per matches
query = """
SELECT
matchid
,MAX(count_rows) AS max_count_rows
FROM (
    SELECT
    matchid
    ,team
    ,roundnumber
    ,COUNT(*) AS count_rows
    FROM r6s_datapeek.dump_rounds
    GROUP BY matchid, team, roundnumber
)
GROUP BY matchid
"""

dfs_match_round_agg = spark.sql(query).persist()
dfs_match_round_agg.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1586459

In [32]:
dfs_matches_stats = dfs_match_stats.join(dfs_match_round_configuration, on = 'matchid').join(dfs_match_round_agg, on = 'matchid').join(dfs_win_match, on = 'matchid').persist()
dfs_matches_stats.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
def compute_average_level(user_levels):
    def compute_average_level_(user_levels):
        levels = [int(elt[1]) for elt in user_levels]
        return float(np.mean(levels))
    return F.udf(compute_average_level_, T.FloatType())(user_levels)

def compute_std_level(user_levels):
    def compute_std_level_(user_levels):
        levels = [int(elt[1]) for elt in user_levels]
        return float(np.std(levels))
    return F.udf(compute_std_level_, T.FloatType())(user_levels)

skillranks = ['Unranked', 'Copper', 'Bronze', 'Silver', 'Gold', 'Platinum', 'Diamond']
def build_count_skillranks(user_levels):
    def build_count_skillranks_(user_levels):
        skillranks_count = [0] * len(skillranks)
        for elt in user_levels:
            skillranks_count[skillranks.index(elt[0])] += 1
        return skillranks_count
    return F.udf(build_count_skillranks_, T.ArrayType(T.IntegerType()))(user_levels)

# Get general statistics at the team level (first round as a reference)
for teamid in range(2):
    dfs_matches_stats = dfs_matches_stats.withColumn(f'team{teamid}_meanlevel', compute_average_level(F.col(f'team{teamid}_user_attributes_firstround')))
    dfs_matches_stats = dfs_matches_stats.withColumn(f'team{teamid}_stdlevel', compute_std_level(F.col(f'team{teamid}_user_attributes_firstround')))
    dfs_matches_stats = dfs_matches_stats.withColumn(f'team{teamid}_skillranks', build_count_skillranks(F.col(f'team{teamid}_user_attributes_firstround')))    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
dfs_matches_stats.sample(False, 0.1, 0).limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      matchid platform  ... team1_stdlevel       team1_skillranks
0  1001541761      PS4  ...      19.126944  [0, 5, 0, 0, 0, 0, 0]
1  1002578761     XONE  ...      22.337412  [0, 5, 0, 0, 0, 0, 0]
2   100343401     XONE  ...      29.357111  [0, 0, 1, 4, 0, 0, 0]
3  1003852761      PS4  ...      17.447063  [0, 5, 0, 0, 0, 0, 0]
4  1004307321      PS4  ...      40.592609  [0, 4, 1, 0, 0, 0, 0]

[5 rows x 28 columns]

In [35]:
def get_euclidian_distance(a, b):
    def get_euclidian_distance_(a, b):
        return float(np.linalg.norm(np.array(a)-np.array(b)))
    return F.udf(get_euclidian_distance_, T.FloatType())(a, b)

# Define a team as reference (in this case team 0)
teamid_reference = 0
teamid_opponent = 1 if teamid_reference == 0 else 0

# Determine the difference on some metrics on the wo teams
dfs_matches_stats = dfs_matches_stats.withColumn('deltateam_meanlevel', F.col(f'team{teamid_reference}_meanlevel') - F.col(f'team{teamid_opponent}_meanlevel'))
dfs_matches_stats = dfs_matches_stats.withColumn('deltateam_stdlevel', F.col(f'team{teamid_reference}_stdlevel') - F.col(f'team{teamid_opponent}_stdlevel'))
dfs_matches_stats = dfs_matches_stats.withColumn('deltateam_skillranks_ed', get_euclidian_distance(F.col(f'team{teamid_reference}_skillranks'), F.col(f'team{teamid_opponent}_skillranks')))
dfs_matches_stats = dfs_matches_stats.withColumn('abs_deltateam_meanlevel', F.abs(F.col('deltateam_meanlevel')))
dfs_matches_stats = dfs_matches_stats.withColumn('abs_deltateam_stdlevel', F.abs(F.col('deltateam_stdlevel')))

# Determine the team who won
dfs_matches_stats = dfs_matches_stats.withColumn('teamreference_won', F.when(F.col(f'team{teamid_reference}_role_count_round_won') > F.col(f'team{teamid_opponent}_role_count_round_won'), 1).otherwise(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Gwt general infos on when the matches occured
dfs_matches_stats = dfs_matches_stats.withColumn('date', F.to_timestamp(F.col('dateid'), 'yyyyMMdd'))
dfs_matches_stats = dfs_matches_stats.withColumn('weekdate', F.date_format(F.col('date'), 'w').cast(T.IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
# Determine the patches that is currently on 
dfs_matches_stats = dfs_matches_stats.withColumn('patch', 
                                                 F.when((F.col('dateid') >= 20170207) & (F.col('dateid') < 20170221), '2.1.0')
                                                 .when((F.col('dateid') >= 20170221) & (F.col('dateid') < 20170315), '2.1.1')
                                                 .when((F.col('dateid') >= 20170315) & (F.col('dateid') < 20170419), '2.1.2').otherwise('2.1.3'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
dfs_matches_stats.sample(False, 0.1, 0).limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      matchid platform     mapname  ...       date  weekdate  patch
0  1001541761      PS4      CHALET  ... 2017-04-22        16  2.1.3
1  1002578761     XONE      OREGON  ... 2017-03-15        11  2.1.2
2   100343401     XONE       HOUSE  ... 2017-05-14        20  2.1.3
3  1003852761      PS4  CLUB HOUSE  ... 2017-05-05        18  2.1.3
4  1004307321      PS4   CONSULATE  ... 2017-05-11        19  2.1.3

[5 rows x 37 columns]

In [48]:
dfs_matches_stats.write.partitionBy('dateid').mode('overwrite').saveAsTable('r6s_datapeek.matches_stats')
print('DONE')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DONE

## Debug

In [50]:
dfs_matches_stats.groupBy(['patch']).agg(F.min('dateid'), F.max('dateid')).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'NoneType' object has no attribute 'groupBy'
Traceback (most recent call last):
AttributeError: 'NoneType' object has no attribute 'groupBy'

